# **♻️ PixelPickup MVP Version 1.0**
 
Welcome to the PixelPickup MVP web interface! This tool allows you to manually select product categories, followed by uploading up to 3 images of your product. The AI model will then evaluate whether such product is eligible for return and then automatically decide whether this will be returned to store, donated, or recycled. Following this decision process, we allow users to input their locations and search for any pick-up drivers nearby. The user will have the option to select who picks up their eligible item for return.

In [ ]:
import requests
import json
import os
import io
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
from google import genai
from google.genai import types
from geopy.geocoders import Nominatim
import folium
import openrouteservice
from dotenv import load_dotenv
from IPython.display import HTML

# We use Gemini and OpenRouteService APIs for this demonstration. We would ideally use Google Maps API if there wasn't a price barrier.
load_dotenv()
ors_key = os.getenv("OPENROUTESERVICE_API_KEY")
gemini_key = os.getenv("GEMINI_API_KEY")

client = openrouteservice.Client(key=ors_key)

# Since this is a demonstration and we do not have real drivers, we will make up the co-ordinates (fixed) for three separate drivers. These are all locations in Toronto.
drivers_coords = {
    "Driver 1": [-79.3906, 43.6582],
    "Driver 2": [-79.3807, 43.6544],
    "Driver 3": [-79.3948, 43.6677]
}

# Output Containers
output_upload = widgets.Output()
output_dropdown = widgets.Output()
output_result = widgets.Output()
output_category = widgets.Output()
output_location = widgets.Output()
output_driver = widgets.Output()

# Set up our Widgets first
# Step 1: Dropdown Widgets (Product Selection)
product_options = ['Topwear', 'Bottomwear', 'Accessories', 'Footwear']
return_reason_options = ['Defective', 'Wrong size', 'Not as described', 'Changed mind']
brand_options = ['Uniqlo', 'Nike', 'Zara', 'Other']
timeline_options = ['Within 30 days', '1-3 months ago', '3-6 months ago', '6+ months ago']
item_condition_options = ['New with tags', 'Gently used', 'Like new', 'Well used']

product_dropdown = widgets.Dropdown(options=product_options, description="Product:")
return_reason_dropdown = widgets.Dropdown(options=return_reason_options, description="Return Reason:")
brand_dropdown = widgets.Dropdown(options=brand_options, description="Brand:")
timeline_dropdown = widgets.Dropdown(options=timeline_options, description="Timeline:")
item_condition_dropdown = widgets.Dropdown(options=item_condition_options, description="Condition:")

assign_button = widgets.Button(description="Confirm Selection", button_style="success")

# Step 2: Image Upload
upload_widget = widgets.FileUpload(accept='image/*', multiple=True)
uploaded_images = {}

# Step 3: Category Selection
category_selection = widgets.RadioButtons(
    options=["Return to Store", "Recycle", "Donate", "Ambiguous"],
    description="Select Category:",
    disabled=True
)
submit_category_button = widgets.Button(description="Submit Selection", button_style="primary")

# Step 4: Location Input
location_input = widgets.Text(description="Your Location:")
submit_location_button = widgets.Button(description="Find Location", button_style="info")

# Step 5: Driver Selection
driver_selection = widgets.RadioButtons(
    options=["Driver 1", "Driver 2", "Driver 3"],
    description="Select Available Driver:",
    disabled=True
)
submit_driver_button = widgets.Button(description="Confirm Driver", button_style="primary")

selected_values = {}

# Function Step 1: Assign Dropdown Values
def assign_values(b):
    selected_values['product'] = product_dropdown.value
    selected_values['return_reason'] = return_reason_dropdown.value
    selected_values['brand'] = brand_dropdown.value
    selected_values['timeline'] = timeline_dropdown.value
    selected_values['item_condition'] = item_condition_dropdown.value

    with output_dropdown:
        clear_output()
        print(f"Product: {selected_values['product']}")
        print(f"Return Reason: {selected_values['return_reason']}")
        print(f"Brand: {selected_values['brand']}")
        print(f"Timeline: {selected_values['timeline']}")
        print(f"Item Condition: {selected_values['item_condition']}")

    # Show upload section
    with output_upload:
        clear_output()
        display(upload_widget)

assign_button.on_click(assign_values)

# Function Step 2: Handle Image Upload
def handle_upload(change):
    global uploaded_images
    uploaded_images.clear()

    with output_upload:
        clear_output()
        for fileinfo in upload_widget.value:
            filename = fileinfo["name"]
            image_data = fileinfo["content"]
            image = Image.open(io.BytesIO(image_data))
            uploaded_images[filename] = image
            display(image)

        print("Images uploaded successfully!")

    # Trigger Gemini processing
    run_gemini_model()

upload_widget.observe(handle_upload, names='value')

# Function Step 3: Process Images with Gemini
def run_gemini_model():
    with output_result:
        clear_output()
        print("Please wait while the AI model evaluates your images and selection criteria...")

    # Process uploaded images for Gemini
    gemini_images = []
    for filename, pil_image in uploaded_images.items():
        if pil_image.mode == "RGBA":
            pil_image = pil_image.convert("RGB")

        img_bytes_io = io.BytesIO()
        pil_image.save(img_bytes_io, format="JPEG")
        img_bytes_io.seek(0)

        gemini_images.append(
            types.Part.from_bytes(
                data=img_bytes_io.getvalue(),
                mime_type="image/jpeg"
            )
        )

    client = genai.Client(api_key=gemini_key)

    response_text = client.models.generate_content(
        model="gemini-2.0-flash-exp",
        contents=[f"""
        Categorize this {selected_values['product']} from {selected_values['brand']}
        (Reason: {selected_values['return_reason']}, Condition: {selected_values['item_condition']}, Timeline: {selected_values['timeline']})
        into:
        1. Return to Store
        2. Recycle
        3. Donate
        First, look at the images and see if they are clothing items; if not, immediately return error. Take into account the brand of the clothing that the user inputted. Otherwise, pretend you are speaking to a customer and provide your reasoning for why you picked the cateogry that you did. If you do not know, just say so. If the product images do not match, say so. Never suggest return to store if product is defective. Also, the categories that the customer inputs may NOT necessarily be correct. Use the images as a higher source of truth.
        """]
    )

    response_numeric = client.models.generate_content(
        model="gemini-2.0-flash-exp",
        contents=[f"""
        Categorize this {selected_values['product']} from {selected_values['brand']}
        (Reason: {selected_values['return_reason']}, Condition: {selected_values['item_condition']}, Timeline: {selected_values['timeline']})
        into:
        1. Return to Store
        2. Recycle
        3. Donate
        Based on {response_text.text}, assign 1-3 to the response and output only that number. If it does not match, then output an error.
        """]
    )

    category_numeric = response_numeric.text.strip()

    category_map = {"1": "Return to Store", "2": "Recycle", "3": "Donate"}
    predicted_category = category_map.get(category_numeric, "Your images do not match each other and the description, are unclear, or ineligble for return. Please upload again.")

    # Display the predicted category and update category selection
    with output_category:
        clear_output()
        print(response_text.text)
        print(f"Based on the AI image classification, the product will undergo the following procedure: {predicted_category}")

    category_selection.value = predicted_category  # Update selected category
    category_selection.disabled = False  # Enable further interaction

    # Once the category is selected, show location input for next step
    with output_location:
        clear_output()
        display(location_input, submit_location_button)

submit_category_button.on_click(run_gemini_model)

# Function Step 4: Get User Location and Geocode it
def get_location(b):
    geolocator = Nominatim(user_agent="geo_finder")
    user_location = location_input.value
    location = geolocator.geocode(user_location)

    if location:
        selected_values['coordinates'] = [location.longitude, location.latitude]
        with output_driver:
            clear_output()
            print(f" Your address: {location.address}")
            # print(f" Your address in lat:long format: ({location.latitude}, {location.longitude})")

            # Calculate distance and time for each driver
            driver_times = {}
            for driver, driver_coords in drivers_coords.items():
                body = {
                    "coordinates": [selected_values['coordinates'], driver_coords],
                    "units": "m"  # metres
                }

                # Headers for ORS API request
                headers = {
                    'Accept': 'application/json, application/geo+json',
                    'Authorization': ors_key,
                    'Content-Type': 'application/json'
                }

                # Make API request to ORS
                url = "https://api.openrouteservice.org/v2/directions/driving-car"
                call = requests.post(url, json=body, headers=headers)

                if call.status_code == 200:
                    response_data = call.json()
                    total_distance = response_data["routes"][0]["summary"]["distance"] / 1000  # Convert meters to km
                    total_duration = response_data["routes"][0]["summary"]["duration"] / 60  # Convert seconds to minutes
                    driver_times[driver] = (total_distance, total_duration)

            # Update driver selection options with distance and time
            driver_selection.options = [
                (f"{driver} ({driver_times[driver][0]:.2f} km, {driver_times[driver][1]:.2f} min)", driver)
                for driver in driver_times
            ]

            display(driver_selection, submit_driver_button)
            driver_selection.disabled = False
    else:
        with output_location:
            clear_output()
            print("Location not found. Try again.")

submit_location_button.on_click(get_location)

# Function Step 5: Assign a Driver and Calculate the Route
def assign_driver(b):
    selected_driver = driver_selection.value
    driver_coords = drivers_coords[selected_driver]

    # Get user's coordinates from selected values
    user_coords = selected_values['coordinates']

    # Prepare the coordinates for the route request
    body = {
        "coordinates": [user_coords, driver_coords],
        "units": "m"  # Get distance in meters
    }

    # Headers for ORS API request
    headers = {
        'Accept': 'application/json, application/geo+json',
        'Authorization': ors_key,
        'Content-Type': 'application/json'
    }

    # Make API request to ORS
    url = "https://api.openrouteservice.org/v2/directions/driving-car"
    call = requests.post(url, json=body, headers=headers)

    if call.status_code == 200:
        response_data = call.json()

        # Extract route data
        total_distance = response_data["routes"][0]["summary"]["distance"] / 1000  # Convert meters to km
        total_duration = response_data["routes"][0]["summary"]["duration"] / 60  # Convert seconds to minutes
        print(f"Driver {selected_driver} assigned!")
        print(f"Distance: {total_distance:.2f} km")
        print(f"Duration: {total_duration:.2f} minutes")

        # Decode polyline geometry for route
        route_geometry = response_data["routes"][0]["geometry"]
        route_coordinates = openrouteservice.convert.decode_polyline(route_geometry)["coordinates"]

        # Create the map with the route
        map_center = [user_coords[1], user_coords[0]]  # lat, lon
        route_map = folium.Map(location=map_center, zoom_start=12)

        # Add the main route polyline to the map
        folium.PolyLine(route_coordinates, color="blue", weight=15, opacity=1.0).add_to(route_map)

        # Add color-coded markers for each location
        locations = [
            user_coords,  # User location
            driver_coords  # Driver location
        ]
        labels = ["Your Location", selected_driver]  # Label for user and driver
        colors = ["blue", "green"]  # Marker colors

        for i, location in enumerate(locations):
            folium.Marker(
                location=[location[1], location[0]],
                popup=labels[i],
                icon=folium.Icon(color=colors[i])
            ).add_to(route_map)

        # Save the map as an HTML file
        route_map.save("route_map_with_driver.html")
        with output_driver:
            clear_output()
            print("Congrats! You have successfully selected a driver and will arrive soon as previously indicated! A map has just been downloaded for you to check your locations :)")
        #     display(widgets.HTML(value='<iframe src="route_map_with_driver.html" width="600" height="400"></iframe>'))
    else:
        print(f"Error: {call.text}")

submit_driver_button.on_click(assign_driver)

# Display UI
display(product_dropdown, return_reason_dropdown, brand_dropdown, timeline_dropdown, item_condition_dropdown, assign_button, output_dropdown, output_upload, output_result, output_category, output_location, output_driver)